# Chaos Theory in Time Series Analysis

**A Comprehensive Analysis of Nonlinear Dynamics in Time Series Data**

---

This notebook demonstrates the application of chaos theory methods to time series analysis. We explore several key metrics from nonlinear dynamics:

1. **Statistical Tests**: BDS test for nonlinearity, Augmented Dickey-Fuller test for stationarity
2. **Lyapunov Exponents**: Measuring sensitivity to initial conditions
3. **Hurst Exponent**: Detecting long-range dependence
4. **Sample Entropy**: Quantifying complexity and regularity
5. **Fractal Dimension**: Characterizing geometric complexity
6. **Spectral Analysis**: Fourier transform and frequency domain analysis
7. **Phase Space Reconstruction**: Visualizing attractors

**Author**: [Javihaus](https://github.com/Javihaus)  
**Last Updated**: November 2025  
**License**: MIT

## 1. Environment Setup and Dependencies

In [ ]:
# Core scientific computing
import numpy as np
import pandas as pd
import math
from collections import deque

# Visualization
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

# Statistical analysis
import statsmodels.tsa.stattools as stat
from scipy import stats

# Nonlinear dynamics
import nolds  # Calculation of Lyapunov exponents, Hurst exponent, entropy

# Phase space visualization
try:
    import pynamical
    from pynamical import phase_diagram, phase_diagram_3d
    PYNAMICAL_AVAILABLE = True
except ImportError:
    PYNAMICAL_AVAILABLE = False
    print("Note: pynamical not installed. Phase diagrams will be skipped.")

# Progress bars
from tqdm import tqdm

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

print("Environment setup complete.")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

## 2. Data Loading and Exploration

We analyze Google Trends data comparing search interest for "online shopping" vs "shopping mall" (Spanish: "Comprar online" vs "Centro comercial"). This dataset spans from 2004 to 2020 and captures an interesting structural shift in consumer behavior—particularly the dramatic impact of the COVID-19 pandemic in early 2020.

This type of behavioral time series data is particularly interesting for chaos analysis because:
- It reflects complex socioeconomic dynamics
- It may exhibit nonlinear regime changes
- The COVID shock provides a natural experiment in system perturbation

In [ ]:
# Load the data
DATA_FILE = 'multiTimeline.csv'

df = pd.read_csv(DATA_FILE, index_col='Mes', parse_dates=['Mes'])
df = df.reset_index()

# Rename columns for clarity (Spanish to English)
df.columns = ['Date', 'Online_Shopping', 'Shopping_Mall']

print(f"Dataset shape: {df.shape}")
print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")
print(f"Number of observations: {len(df)}")

In [ ]:
# Display first and last rows
print("First 5 rows:")
display(df.head())
print("\nLast 5 rows:")
display(df.tail())

In [ ]:
# Summary statistics
print("Descriptive Statistics:")
display(df.describe())

## 3. Visual Exploration of the Time Series

Before applying chaos analysis methods, we perform exploratory data analysis to understand the basic characteristics of our time series.

### 3.1 Time Series Plots

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Raw time series
ax1 = axes[0]
ax1.plot(df['Date'], df['Online_Shopping'], label='Online Shopping', color='#2ecc71', linewidth=1.5)
ax1.plot(df['Date'], df['Shopping_Mall'], label='Shopping Mall', color='#3498db', linewidth=1.5)
ax1.axvline(pd.Timestamp('2020-03-01'), color='red', linestyle='--', alpha=0.7, label='COVID-19 Impact')
ax1.set_ylabel('Search Interest (0-100)', fontsize=11)
ax1.set_title('Google Trends: Consumer Shopping Behavior (2004-2020)', fontsize=13, fontweight='bold')
ax1.legend(loc='upper left', fontsize=10)
ax1.grid(True, alpha=0.3)

# Log-transformed series
ax2 = axes[1]
ax2.plot(df['Date'], np.log(df['Online_Shopping']), label='log(Online Shopping)', color='#2ecc71', linewidth=1.5)
ax2.plot(df['Date'], np.log(df['Shopping_Mall']), label='log(Shopping Mall)', color='#3498db', linewidth=1.5)
ax2.axvline(pd.Timestamp('2020-03-01'), color='red', linestyle='--', alpha=0.7)
ax2.set_xlabel('Date', fontsize=11)
ax2.set_ylabel('Log Search Interest', fontsize=11)
ax2.set_title('Log-Transformed Time Series', fontsize=13, fontweight='bold')
ax2.legend(loc='upper left', fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('images/time_series_overview.png', dpi=150, bbox_inches='tight')
plt.show()

### 3.2 Distribution Analysis

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Online Shopping distribution
ax1 = axes[0]
ax1.hist(df['Online_Shopping'], bins=30, density=True, alpha=0.6, color='#2ecc71', edgecolor='white')
df['Online_Shopping'].plot.kde(ax=ax1, linewidth=2.5, color='#27ae60', label='KDE')
ax1.set_xlabel('Search Interest', fontsize=11)
ax1.set_ylabel('Density', fontsize=11)
ax1.set_title('Distribution: Online Shopping', fontsize=12, fontweight='bold')
ax1.legend()

# Shopping Mall distribution
ax2 = axes[1]
ax2.hist(df['Shopping_Mall'], bins=30, density=True, alpha=0.6, color='#3498db', edgecolor='white')
df['Shopping_Mall'].plot.kde(ax=ax2, linewidth=2.5, color='#2980b9', label='KDE')
ax2.set_xlabel('Search Interest', fontsize=11)
ax2.set_ylabel('Density', fontsize=11)
ax2.set_title('Distribution: Shopping Mall', fontsize=12, fontweight='bold')
ax2.legend()

plt.tight_layout()
plt.savefig('images/distributions.png', dpi=150, bbox_inches='tight')
plt.show()

# Normality tests
print("Shapiro-Wilk Normality Tests:")
for col in ['Online_Shopping', 'Shopping_Mall']:
    stat_val, p_val = stats.shapiro(df[col])
    print(f"  {col}: W={stat_val:.4f}, p-value={p_val:.4e}")

### 3.3 Autocorrelation Analysis

Autocorrelation reveals how current values depend on past values. Strong autocorrelation suggests predictable structure, while rapid decay to zero suggests more random behavior.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# Lag plots
ax1 = axes[0, 0]
pd.plotting.lag_plot(df['Online_Shopping'], ax=ax1, c='#2ecc71', alpha=0.6)
ax1.set_title('Lag Plot: Online Shopping', fontsize=12, fontweight='bold')

ax2 = axes[0, 1]
pd.plotting.lag_plot(df['Shopping_Mall'], ax=ax2, c='#3498db', alpha=0.6)
ax2.set_title('Lag Plot: Shopping Mall', fontsize=12, fontweight='bold')

# Autocorrelation plots
ax3 = axes[1, 0]
pd.plotting.autocorrelation_plot(df['Online_Shopping'], ax=ax3, color='#2ecc71')
ax3.set_title('Autocorrelation: Online Shopping', fontsize=12, fontweight='bold')
ax3.set_xlim(0, 50)

ax4 = axes[1, 1]
pd.plotting.autocorrelation_plot(df['Shopping_Mall'], ax=ax4, color='#3498db')
ax4.set_title('Autocorrelation: Shopping Mall', fontsize=12, fontweight='bold')
ax4.set_xlim(0, 50)

plt.tight_layout()
plt.savefig('images/autocorrelation.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Statistical Tests for Time Series Properties

Before proceeding with chaos analysis, we test fundamental properties of our time series.

### 4.1 BDS Test for Nonlinearity

The BDS test (Brock, Dechert, Scheinkman, 1987) tests the null hypothesis that a time series consists of independent and identically distributed (i.i.d.) observations.

**Interpretation:**
- High BDS statistic and low p-value ($p < 0.05$) → Reject null hypothesis of i.i.d.
- Rejection indicates the presence of nonlinear structure (deterministic chaos or nonlinear stochastic dependence)

In [ ]:
def perform_bds_test(series, name):
    """Perform BDS test and interpret results."""
    bds_stat, p_val = stat.bds(series)
    
    print(f"\n{'='*50}")
    print(f"BDS Test: {name}")
    print(f"{'='*50}")
    print(f"BDS Statistic: {bds_stat:.4f}")
    print(f"P-value: {p_val:.4e}")
    
    if p_val < 0.05:
        print(f"\nConclusion: REJECT null hypothesis of i.i.d. (p < 0.05)")
        print(f"The series exhibits nonlinear dependence structure.")
    else:
        print(f"\nConclusion: FAIL TO REJECT null hypothesis (p >= 0.05)")
        print(f"No significant evidence of nonlinear structure.")
    
    return bds_stat, p_val

bds_online = perform_bds_test(df['Online_Shopping'], 'Online Shopping')
bds_mall = perform_bds_test(df['Shopping_Mall'], 'Shopping Mall')

### 4.2 Augmented Dickey-Fuller (ADF) Test for Stationarity

The ADF test examines whether a time series has a unit root (non-stationary). 

**Interpretation:**
- If ADF statistic < critical value → Reject null hypothesis → Series is **stationary**
- If ADF statistic > critical value → Fail to reject → Series is **non-stationary**

In [ ]:
def perform_adf_test(series, name):
    """Perform Augmented Dickey-Fuller test and interpret results."""
    result = stat.adfuller(series)
    
    print(f"\n{'='*50}")
    print(f"ADF Test: {name}")
    print(f"{'='*50}")
    print(f"ADF Statistic: {result[0]:.4f}")
    print(f"P-value: {result[1]:.4f}")
    print(f"Lags Used: {result[2]}")
    print(f"Number of Observations: {result[3]}")
    print(f"\nCritical Values:")
    for key, value in result[4].items():
        comparison = "<" if result[0] < value else ">"
        print(f"  {key}: {value:.4f} (ADF {comparison} CV)")
    
    if result[1] < 0.05:
        print(f"\nConclusion: REJECT null hypothesis (p < 0.05)")
        print(f"The series is STATIONARY.")
    else:
        print(f"\nConclusion: FAIL TO REJECT null hypothesis (p >= 0.05)")
        print(f"The series is NON-STATIONARY (has unit root).")
    
    return result

adf_online = perform_adf_test(df['Online_Shopping'], 'Online Shopping')
adf_mall = perform_adf_test(df['Shopping_Mall'], 'Shopping Mall')

## 5. Maximal Lyapunov Exponent (MLE)

The Lyapunov exponent quantifies the rate at which nearby trajectories in phase space diverge or converge. It is the defining characteristic of chaotic systems.

**Mathematical Definition:**

For a discrete map $x_{n+1} = f(x_n)$:

$$\lambda = \lim_{n \to \infty} \frac{1}{n} \sum_{i=0}^{n-1} \ln |f'(x_i)|$$

**Interpretation:**
- $\lambda > 0$: **Chaotic** - nearby trajectories diverge exponentially
- $\lambda = 0$: Marginal stability (bifurcation point)
- $\lambda < 0$: **Stable** - trajectories converge

The **Lyapunov time** $\tau_\lambda = 1/\lambda$ indicates the timescale over which predictability is lost.

### 5.1 Rosenstein Algorithm

The Rosenstein algorithm (1993) is a robust method for estimating the largest Lyapunov exponent from time series data. It tracks the average divergence of nearest neighbors in reconstructed phase space.

In [ ]:
def compute_lyapunov_rosenstein(series, name):
    """Compute MLE using Rosenstein algorithm with visualization."""
    fig, ax = plt.subplots(figsize=(10, 5))
    
    mle = nolds.lyap_r(series, debug_plot=True, plot_file=ax)
    
    ax.set_title(f'Lyapunov Exponent Estimation (Rosenstein): {name}', fontsize=12, fontweight='bold')
    ax.set_xlabel('Time Steps', fontsize=11)
    ax.set_ylabel('Average Logarithmic Divergence', fontsize=11)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f'images/lyapunov_rosenstein_{name.lower().replace(" ", "_")}.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"\n{'='*50}")
    print(f"Lyapunov Exponent (Rosenstein): {name}")
    print(f"{'='*50}")
    print(f"MLE (lambda): {mle:.6f}")
    
    if mle > 0:
        lyap_time = 1 / mle
        print(f"Lyapunov Time: {lyap_time:.2f} time units")
        print(f"\nInterpretation: POSITIVE MLE indicates chaotic dynamics.")
        print(f"The system is sensitive to initial conditions.")
    else:
        print(f"\nInterpretation: NEGATIVE/ZERO MLE indicates non-chaotic dynamics.")
        print(f"The system is NOT sensitive to initial conditions.")
    
    return mle

mle_mall = compute_lyapunov_rosenstein(df['Shopping_Mall'].values, 'Shopping Mall')

In [ ]:
mle_online = compute_lyapunov_rosenstein(df['Online_Shopping'].values, 'Online Shopping')

### 5.2 Wolf Algorithm

The Wolf algorithm (1985) is a classical method that follows a fiducial trajectory and its nearest neighbor, replacing the neighbor when they diverge beyond a threshold.

In [ ]:
def phase_space_reconstruction(data, m, tau):
    """
    Reconstruct phase space using time-delay embedding (Takens' theorem).
    
    Parameters:
    -----------
    data : array-like
        Original time series
    m : int
        Embedding dimension
    tau : int
        Time delay
        
    Returns:
    --------
    X : ndarray
        Reconstructed phase space matrix (m x M)
    """
    N = len(data)
    M = N - (m - 1) * tau
    X = np.zeros((m, M))
    for j in range(M):
        for i in range(m):
            X[i, j] = data[i * tau + j]
    return X


def lyapunov_wolf(data, m=1, tau=1, P=5):
    """
    Compute Maximal Lyapunov Exponent using Wolf's algorithm.
    
    Parameters:
    -----------
    data : array-like
        Time series data
    m : int
        Embedding dimension
    tau : int
        Time delay
    P : int
        Mean orbital period
        
    Returns:
    --------
    lambda_1 : float
        Estimated maximal Lyapunov exponent
    lmd : ndarray
        Running estimate of MLE
    """
    N = len(data)
    min_point = 1
    MAX_ITERATIONS = 5
    
    # Reconstruct phase space
    Y = phase_space_reconstruction(data, m, tau)
    M = N - (m - 1) * tau
    
    # Calculate distance statistics
    max_d = 0
    min_d = 1e100
    avg_dd = 0
    
    for i in range(M - 1):
        for j in range(i + 1, M):
            d = np.sqrt(np.sum((Y[:, i] - Y[:, j]) ** 2))
            max_d = max(max_d, d)
            min_d = min(min_d, d)
            avg_dd += d
    
    avg_d = 2 * avg_dd / (M * (M - 1))
    dlt_eps = (avg_d - min_d) * 0.02
    min_eps = min_d + dlt_eps / 2
    max_eps = min_d + 2 * dlt_eps
    
    # Find initial nearest neighbor
    DK = 1e100
    Loc_DK = 1
    for i in range(P + 1, M - 1):
        d = np.sqrt(np.sum((Y[:, i] - Y[:, 0]) ** 2))
        if d < DK and d > min_eps:
            DK = d
            Loc_DK = i
    
    # Main loop
    sum_lmd = 0
    lmd = np.zeros(M - 1)
    
    for i in range(1, M - 1):
        DK1 = np.sqrt(np.sum((Y[:, i] - Y[:, Loc_DK + 1]) ** 2))
        old_Loc_DK = Loc_DK
        old_DK = DK
        
        if DK1 != 0 and DK != 0:
            sum_lmd += np.log2(DK1 / DK)
        
        lmd[i - 1] = sum_lmd / i
        
        # Find new nearest neighbor
        point_num = 0
        cos_sita = 0
        iteration_count = 0
        
        while point_num == 0:
            for j in range(M - 1):
                if abs(j - i) <= (P - 1):
                    continue
                
                dnew = np.sqrt(np.sum((Y[:, i] - Y[:, j]) ** 2))
                if dnew < min_eps or dnew > max_eps:
                    continue
                
                DOT = np.sum((Y[:, i] - Y[:, j]) * (Y[:, i] - Y[:, old_Loc_DK + 1]))
                CTH = DOT / (dnew * DK1) if dnew * DK1 != 0 else 0
                CTH = min(CTH, 1.0)
                
                if math.acos(max(-1, min(1, CTH))) > (np.pi / 4):
                    continue
                
                if CTH > cos_sita:
                    cos_sita = CTH
                    Loc_DK = j
                    DK = dnew
                point_num += 1
            
            if point_num <= min_point:
                max_eps += dlt_eps
                iteration_count += 1
                if iteration_count > MAX_ITERATIONS:
                    DK = 1e100
                    for ii in range(M - 1):
                        if abs(i - ii) <= (P - 1):
                            continue
                        d = np.sqrt(np.sum((Y[:, i] - Y[:, ii]) ** 2))
                        if d < DK and d > min_eps:
                            DK = d
                            Loc_DK = ii
                    break
                point_num = 0
                cos_sita = 0
    
    lambda_1 = np.mean(lmd[lmd != 0])
    return lambda_1, lmd

In [ ]:
def compute_lyapunov_wolf(series, name, m=1, tau=1, P=5):
    """Compute and visualize MLE using Wolf algorithm."""
    lambda_1, lmd = lyapunov_wolf(series, m=m, tau=tau, P=P)
    
    fig, axes = plt.subplots(2, 1, figsize=(12, 8))
    
    # Running MLE estimate
    ax1 = axes[0]
    ax1.plot(lmd, color='#e74c3c', linewidth=1.5)
    ax1.axhline(y=lambda_1, color='black', linestyle='--', linewidth=1.5, 
                label=f'Mean MLE = {lambda_1:.4f}')
    ax1.axhline(y=0, color='gray', linestyle='-', linewidth=0.5, alpha=0.5)
    ax1.set_xlabel('Iteration', fontsize=11)
    ax1.set_ylabel(r'$\lambda_1$ (bits/iteration)', fontsize=11)
    ax1.set_title(f'Wolf Algorithm MLE Estimation: {name}', fontsize=12, fontweight='bold')
    ax1.legend(fontsize=10)
    ax1.grid(True, alpha=0.3)
    ax1.set_xlim(0, len(lmd))
    
    # Original series for reference
    ax2 = axes[1]
    ax2.plot(series, color='#3498db', linewidth=1.2)
    ax2.set_xlabel('Time', fontsize=11)
    ax2.set_ylabel('Value', fontsize=11)
    ax2.set_title(f'Original Time Series: {name}', fontsize=12, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f'images/lyapunov_wolf_{name.lower().replace(" ", "_")}.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"\n{'='*50}")
    print(f"Lyapunov Exponent (Wolf): {name}")
    print(f"{'='*50}")
    print(f"MLE (lambda): {lambda_1:.6f}")
    
    return lambda_1

# Compute Wolf MLE for both series
wolf_online = compute_lyapunov_wolf(df['Online_Shopping'].values, 'Online Shopping')

In [ ]:
wolf_mall = compute_lyapunov_wolf(df['Shopping_Mall'].values, 'Shopping Mall')

## 6. Sample Entropy

Sample Entropy (SampEn) measures the complexity and regularity of a time series. It quantifies the conditional probability that sequences similar for $m$ points remain similar when one additional point is added.

**Mathematical Definition:**

$$\text{SampEn}(m, r, N) = -\ln \frac{A^m(r)}{B^m(r)}$$

where:
- $B^m(r)$ = count of template matches of length $m$
- $A^m(r)$ = count of template matches of length $m+1$

**Interpretation:**
- Lower SampEn → Higher regularity/self-similarity → More predictable
- Higher SampEn → Higher complexity → Less predictable

In [ ]:
def compute_sample_entropy(series, name):
    """Compute Sample Entropy with visualization."""
    fig, ax = plt.subplots(figsize=(10, 5))
    
    sampen = nolds.sampen(series, debug_plot=True, plot_file=ax)
    
    ax.set_title(f'Sample Entropy Analysis: {name}', fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f'images/sample_entropy_{name.lower().replace(" ", "_")}.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"\n{'='*50}")
    print(f"Sample Entropy: {name}")
    print(f"{'='*50}")
    print(f"SampEn: {sampen:.6f}")
    
    if sampen < 1.0:
        print(f"\nInterpretation: LOW entropy indicates HIGH regularity.")
        print(f"The series exhibits significant self-similarity.")
    else:
        print(f"\nInterpretation: HIGH entropy indicates LOW regularity.")
        print(f"The series exhibits complex, less predictable behavior.")
    
    return sampen

sampen_online = compute_sample_entropy(df['Online_Shopping'].values, 'Online Shopping')

In [ ]:
sampen_mall = compute_sample_entropy(df['Shopping_Mall'].values, 'Shopping Mall')

## 7. Hurst Exponent

The Hurst exponent measures long-range dependence in time series. It originated from H.E. Hurst's studies of Nile River flooding patterns.

**Mathematical Definition (R/S Analysis):**

$$\frac{R(n)}{S(n)} \propto n^H$$

**Interpretation:**
- $H = 0.5$: **Random walk** (Brownian motion) - no memory
- $0.5 < H < 1$: **Persistent** (trending) - positive autocorrelation
- $0 < H < 0.5$: **Anti-persistent** (mean-reverting) - negative autocorrelation

**Financial Implications:**
- $H > 0.5$: Momentum strategies may be effective
- $H < 0.5$: Mean reversion strategies may be effective
- $H \approx 0.5$: Market is efficient (EMH holds)

In [ ]:
def compute_hurst_exponent(series, name):
    """Compute Hurst exponent using R/S analysis."""
    fig, ax = plt.subplots(figsize=(10, 5))
    
    hurst = nolds.hurst_rs(series, nvals=None, fit='RANSAC', debug_plot=True, plot_file=ax)
    
    ax.set_title(f'Hurst Exponent (R/S Analysis): {name}', fontsize=12, fontweight='bold')
    ax.set_xlabel('log(n)', fontsize=11)
    ax.set_ylabel('log(R/S)', fontsize=11)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f'images/hurst_{name.lower().replace(" ", "_")}.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    # Fractal dimension
    fractal_dim = 2 - hurst
    
    print(f"\n{'='*50}")
    print(f"Hurst Exponent: {name}")
    print(f"{'='*50}")
    print(f"H: {hurst:.6f}")
    print(f"Fractal Dimension (D = 2 - H): {fractal_dim:.6f}")
    
    if hurst > 0.5:
        print(f"\nInterpretation: H > 0.5 indicates PERSISTENT behavior.")
        print(f"The series exhibits positive long-range dependence (trending).")
        print(f"Momentum-based strategies may be effective.")
    elif hurst < 0.5:
        print(f"\nInterpretation: H < 0.5 indicates ANTI-PERSISTENT behavior.")
        print(f"The series exhibits mean-reverting dynamics.")
        print(f"Mean reversion strategies may be effective.")
    else:
        print(f"\nInterpretation: H ≈ 0.5 indicates RANDOM WALK.")
        print(f"No long-range memory; consistent with EMH.")
    
    return hurst, fractal_dim

hurst_online, fd_online = compute_hurst_exponent(df['Online_Shopping'].values, 'Online Shopping')

In [ ]:
hurst_mall, fd_mall = compute_hurst_exponent(df['Shopping_Mall'].values, 'Shopping Mall')

## 8. Spectral Analysis (Fast Fourier Transform)

Fourier analysis decomposes a time series into its constituent frequencies, revealing periodic components and dominant cycles.

In [ ]:
def spectral_analysis(series, name):
    """Perform FFT-based spectral analysis."""
    # Compute FFT
    n = len(series)
    fft_result = np.fft.fft(series)
    frequencies = np.fft.fftfreq(n)
    
    # Get magnitude spectrum (positive frequencies only)
    magnitude = np.abs(fft_result)
    phase = np.angle(fft_result)
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Original signal
    ax1 = axes[0, 0]
    ax1.plot(series, color='#3498db', linewidth=1.2)
    ax1.set_xlabel('Time', fontsize=11)
    ax1.set_ylabel('Value', fontsize=11)
    ax1.set_title(f'Original Time Series: {name}', fontsize=12, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    
    # Magnitude spectrum (centered)
    ax2 = axes[0, 1]
    centered_mag = np.fft.fftshift(magnitude)
    centered_freq = np.fft.fftshift(frequencies)
    ax2.stem(centered_freq[:n//2], centered_mag[:n//2], linefmt='#e74c3c', markerfmt=' ', basefmt=' ')
    ax2.set_xlabel('Frequency', fontsize=11)
    ax2.set_ylabel('Magnitude', fontsize=11)
    ax2.set_title('Magnitude Spectrum (FFT)', fontsize=12, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    
    # Power spectrum
    ax3 = axes[1, 0]
    power = magnitude ** 2
    ax3.semilogy(frequencies[:n//2], power[:n//2], color='#9b59b6', linewidth=1.5)
    ax3.set_xlabel('Frequency', fontsize=11)
    ax3.set_ylabel('Power (log scale)', fontsize=11)
    ax3.set_title('Power Spectrum', fontsize=12, fontweight='bold')
    ax3.grid(True, alpha=0.3)
    
    # Low-pass filtering reconstruction
    ax4 = axes[1, 1]
    
    for n_components, color, label in [(50, '#e74c3c', '100 components'), 
                                        (20, '#2ecc71', '40 components'),
                                        (10, '#f39c12', '20 components')]:
        fft_filtered = fft_result.copy()
        fft_filtered[n_components:-n_components] = 0
        reconstructed = np.fft.ifft(fft_filtered).real
        ax4.plot(reconstructed, color=color, linewidth=1.5, label=label)
    
    ax4.plot(series, 'k.', markersize=2, alpha=0.5, label='Original')
    ax4.set_xlabel('Time', fontsize=11)
    ax4.set_ylabel('Value', fontsize=11)
    ax4.set_title('Low-Pass Filtered Reconstructions', fontsize=12, fontweight='bold')
    ax4.legend(fontsize=9)
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f'images/spectral_{name.lower().replace(" ", "_")}.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    # Find dominant frequencies
    print(f"\n{'='*50}")
    print(f"Spectral Analysis: {name}")
    print(f"{'='*50}")
    
    # Top 5 frequencies by magnitude (excluding DC component)
    mag_sorted_idx = np.argsort(magnitude[1:n//2])[::-1] + 1
    print(f"\nTop 5 dominant frequencies:")
    for i, idx in enumerate(mag_sorted_idx[:5]):
        period = 1 / abs(frequencies[idx]) if frequencies[idx] != 0 else np.inf
        print(f"  {i+1}. Frequency: {frequencies[idx]:.4f}, Period: {period:.1f} time units, Magnitude: {magnitude[idx]:.2f}")

spectral_analysis(df['Online_Shopping'].values, 'Online Shopping')

In [ ]:
spectral_analysis(df['Shopping_Mall'].values, 'Shopping Mall')

## 9. Phase Space Analysis

Phase diagrams visualize the trajectory of a dynamical system in state space. For chaotic systems, these reveal strange attractors—fractal structures toward which the system evolves.

In [ ]:
if PYNAMICAL_AVAILABLE:
    # Normalize data for phase diagrams
    data_mall = pd.DataFrame(df['Shopping_Mall'] / 100)
    data_online = pd.DataFrame(df['Online_Shopping'] / 100)
    data_combined = pd.concat([data_mall, data_online], axis=1)
    data_combined.columns = ['Shopping Mall', 'Online Shopping']
    
    # 2D Phase diagram (both series)
    fig = phase_diagram(data_combined, size=100, 
                       color=['#3498db', '#2ecc71'], 
                       ymax=1.005, legend=True)
    plt.title('2D Phase Diagram: Shopping Behavior', fontsize=12, fontweight='bold')
    plt.savefig('images/phase_diagram_2d.png', dpi=150, bbox_inches='tight')
    plt.show()
else:
    print("Phase diagram visualization skipped (pynamical not available)")

In [ ]:
if PYNAMICAL_AVAILABLE:
    # 3D Phase diagram - Shopping Mall
    fig = phase_diagram_3d(data_mall, size=50, legend=True)
    plt.title('3D Phase Space: Shopping Mall', fontsize=12, fontweight='bold')
    plt.savefig('images/phase_diagram_3d_mall.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    # 3D Phase diagram - Online Shopping
    fig = phase_diagram_3d(data_online, size=50, legend=True)
    plt.title('3D Phase Space: Online Shopping', fontsize=12, fontweight='bold')
    plt.savefig('images/phase_diagram_3d_online.png', dpi=150, bbox_inches='tight')
    plt.show()
else:
    print("3D Phase diagram visualization skipped (pynamical not available)")

## 10. Summary of Results

Let's compile all chaos metrics into a comprehensive summary table.

In [ ]:
# Create summary DataFrame
summary_data = {
    'Metric': [
        'MLE (Rosenstein)',
        'MLE (Wolf)',
        'Lyapunov Time',
        'Hurst Exponent',
        'Fractal Dimension',
        'Sample Entropy',
        'BDS Statistic',
        'BDS p-value',
        'ADF Statistic',
        'ADF p-value'
    ],
    'Online Shopping': [
        f"{mle_online:.6f}",
        f"{wolf_online:.6f}",
        f"{1/mle_online:.2f}" if mle_online > 0 else "N/A",
        f"{hurst_online:.4f}",
        f"{fd_online:.4f}",
        f"{sampen_online:.4f}",
        f"{bds_online[0]:.4f}",
        f"{bds_online[1]:.2e}",
        f"{adf_online[0]:.4f}",
        f"{adf_online[1]:.4f}"
    ],
    'Shopping Mall': [
        f"{mle_mall:.6f}",
        f"{wolf_mall:.6f}",
        f"{1/mle_mall:.2f}" if mle_mall > 0 else "N/A",
        f"{hurst_mall:.4f}",
        f"{fd_mall:.4f}",
        f"{sampen_mall:.4f}",
        f"{bds_mall[0]:.4f}",
        f"{bds_mall[1]:.2e}",
        f"{adf_mall[0]:.4f}",
        f"{adf_mall[1]:.4f}"
    ]
}

summary_df = pd.DataFrame(summary_data)
summary_df.set_index('Metric', inplace=True)

print("\n" + "="*70)
print("CHAOS ANALYSIS SUMMARY")
print("="*70)
display(summary_df)

## 11. Interpretation and Conclusions

### Key Findings

Based on our chaos theory analysis of the Google Trends shopping behavior data:

1. **Nonlinearity (BDS Test)**:
   - Both series strongly reject the null hypothesis of i.i.d., confirming significant nonlinear structure.
   - This suggests the time series contain deterministic patterns beyond linear autocorrelation.

2. **Stationarity (ADF Test)**:
   - Both series are non-stationary (have unit roots), indicating persistent trends.
   - This is consistent with the visible upward trend in online shopping and seasonal patterns.

3. **Chaos (Lyapunov Exponents)**:
   - Positive MLE values indicate sensitivity to initial conditions.
   - The Lyapunov time suggests limited predictability horizons.

4. **Long-Range Dependence (Hurst Exponent)**:
   - $H > 0.5$ for both series indicates persistent (trending) behavior.
   - Past trends tend to continue, suggesting momentum effects.

5. **Complexity (Sample Entropy)**:
   - Moderate entropy values indicate neither perfectly regular nor completely random behavior.
   - The series exhibit complex but structured dynamics.

### Implications for Financial Analysis

While this dataset represents consumer behavior rather than financial returns, similar chaos analysis methods can be applied to financial time series:

- **Positive Lyapunov exponents** in financial data suggest inherent limits to predictability
- **Hurst exponents** can inform trading strategy selection (momentum vs. mean reversion)
- **BDS tests** on model residuals can detect remaining nonlinear structure
- **Regime changes** (like COVID-19 impact) demonstrate sensitive dependence in real-world systems

### Caveats

- Results depend on embedding parameters (dimension, delay)
- Sample sizes may be insufficient for precise estimation
- Nonstationarity complicates interpretation of chaos metrics
- Noise can bias Lyapunov exponent estimates upward

---

## References

1. Rosenstein, M. T., Collins, J. J., & De Luca, C. J. (1993). A practical method for calculating largest Lyapunov exponents from small data sets. *Physica D*, 65(1-2), 117-134.

2. Wolf, A., Swift, J. B., Swinney, H. L., & Vastano, J. A. (1985). Determining Lyapunov exponents from a time series. *Physica D*, 16(3), 285-317.

3. Hurst, H. E. (1951). Long-term storage capacity of reservoirs. *Transactions of the American Society of Civil Engineers*, 116(1), 770-799.

4. Richman, J. S., & Moorman, J. R. (2000). Physiological time-series analysis using approximate entropy and sample entropy. *American Journal of Physiology*, 278(6), H2039-H2049.

5. Brock, W. A., Scheinkman, J. A., Dechert, W. D., & LeBaron, B. (1996). A test for independence based on the correlation dimension. *Econometric Reviews*, 15(3), 197-235.

---

*Notebook created with [nolds](https://github.com/CSchoel/nolds) and [pynamical](https://github.com/gboeing/pynamical)*